In [ ]:
from google.colab import drive
import glob, os
from PIL import Image

# Mount Google Drive
drive.mount('/content/drive')

# # Set the image folder path (adjust if needed)
# image_folder = "/content/drive/MyDrive/BA820-Unsupervised ML/BA 820 Team 6/milestone 2/MarketingCharts"

# # Retrieve list of image files in the folder (adjust file pattern as needed)
# image_paths = glob.glob(os.path.join(image_folder, "*.*"))
# print(f"Found {len(image_paths)} images in the folder.")
# for path in image_paths:
#     print(os.path.basename(path))

Mounted at /content/drive


In [ ]:
import base64

image_path = "/content/drive/MyDrive/BA820-Unsupervised ML/BA 820 Team 6/milestone 2/MarketingCharts/scatter-line.png"  # update this path

# Read the image in binary and encode to base64
with open(image_path, "rb") as img_file:
    image_bytes = img_file.read()
    base64_image = base64.b64encode(image_bytes).decode('utf-8')

# Prepare a data URL string with the appropriate MIME type (e.g., image/png or image/jpeg)
mime_type = "image/png"  # change to "image/jpeg" if your image is a JPG
data_url = f"data:{mime_type};base64,{base64_image}"


In [ ]:
from google.colab import userdata
from openai import OpenAI
import openai
openai.api_key = userdata.get('openai')  # OpenAI API key for ChatGPT
client = OpenAI(api_key=openai.api_key)

###Step 0: Initiate the task

In [ ]:
# DO NOT CHANGE THIS PART!!!
# NEVER EVER

import openai

openai.api_key = openai.api_key  # ensure your API key is set

# Create the message with text prompt + image
messages = [
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "Please analyze the following chart and provide insights:"},
            {"type": "image_url", "image_url": {"url": data_url}}
        ]
    }
]

# Send the request to the GPT-4o-mini model
response = client.chat.completions.create(
    model="gpt-4o-mini",       # or another vision-enabled model you have access to
    messages=messages,
    max_tokens=500             # adjust as needed for a full answer
)

# Extract and print the model's answer
answer = response.choices[0].message.content
print(answer)

The chart presented is a control chart showing the relationship between total orders and late packaging incidents. Here are some insights based on the visual data:

1. **Trend Analysis**: The overall trend appears to be a downward slope, indicating that as total orders increase, the number of late packaging incidents tends to decrease. This suggests possible improvements in the packaging process or enhanced efficiency over time.

2. **Variability**: There are fluctuations represented by the scattered points around the trend line. While late packaging incidents decline overall, there are instances where late deliveries spike, particularly around certain total order values (e.g., around 336 and 321). This variability may indicate specific issues that warrant further investigation.

3. **Performance Indicators**: The highest points on the chart indicate instances of late packaging exceeding 50 incidents, especially at lower total order numbers. This suggests that smaller orders might stil

In [ ]:
chatgpt_model = 'gpt-4o-mini'

In [ ]:
# DO NOT CHANGE THIS PART!!!
# NEVER EVER
# Please analyze the following chart and provide one key business insight. "
#         "Assume the user is a U.S.-based small business owner with at most some college education. "
#         "Use plain, easy-to-understand English and avoid technical jargon.

def get_chatgpt_insight_from_dataurl(data_url):
    # Combine your persona note with the analysis prompt
    prompt_text = (
        "Help me to understand the chart"
    )

    # Create the message with structured content: a text part and an image part.
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": prompt_text},
                {"type": "image_url", "image_url": {"url": data_url}}
            ]
        }
    ]

    try:
        chat_response = client.chat.completions.create(
            model=chatgpt_model,
            messages=messages,
            max_tokens=500  # adjust as needed for a full answer
        )
        chat_text = chat_response.choices[0].message.content.strip()
    except Exception as e:
        chat_text = f"Error: {e}"
    return chat_text

# Example usage: Pass the pre-encoded data_url to the function
print(get_chatgpt_insight_from_dataurl(data_url))

The chart you provided is a packaging control chart that displays the relationship between the "Total Order" (on the x-axis) and "Late Packaging" (on the y-axis).

Here’s a breakdown to help you understand it:

1. **Axes**:
   - **X-Axis (Total Order)**: Represents the total number of orders. As you move from left to right, the number of orders increases.
   - **Y-Axis (Late Packaging)**: Represents the number of instances of late packaging. This measurement indicates how many orders were packaged late.

2. **Data Points**: The chart has several red dots that represent the number of late packaging instances for various total orders. These points appear scattered, indicating variability in the late packaging occurrences.

3. **Trend Line**: The straight line running through the data points suggests a downward trend, indicating that as the total number of orders increases, the instances of late packaging tend to decrease overall. 

4. **Analysis**: 
   - **Overall Decrease**: This could 

###Step 1: Define the Evaluation Function

In [ ]:
import re
import json

def evaluate_insight(insight_text):
    eval_prompt = f"""
Please evaluate the following insight according to these criteria. Provide a rating (0 to 5) for each category in JSON format:
Accuracy (5: Fully accurate identification of the chart type and key data insights. 3-4: Minor errors but generally correct. 1-2: Significant inaccuracies or missing key details. 0: Completely incorrect),
Relevance (5: Highlights the most critical insights with no unnecessary details. 3-4: Covers relevant points but includes minor off-topic information. 1-2: Misses key insights or includes too much irrelevant detail. 0: Completely irrelevant),
Clarity (5: Very easy to understand with no jargon. 3-4: Mostly clear but has some complex wording. 1-2: Hard to understand due to technical terms or vague phrasing. 0: Unclear or confusing),
Actionability (5: Clear, practical actions that are directly useful. 3-4: Somewhat useful but could be more specific. 1-2: Actions are too vague or impractical. 0: No actionable insights),
Conciseness (5: Under 50 words, concise, and to the point. 3-4: Slightly over 50 words but still clear. 1-2: Too wordy or contains unnecessary details. 0: Far exceeds word limit or is too brief to be meaningful).

Insight: {insight_text}

Please output the result as:
{{"Accuracy": x, "Relevance": x, "Clarity": x, "Actionability": x, "Conciseness": x}}
    """
    chat_messages = [
        {"role": "user", "content": eval_prompt}
    ]

    try:
        chat_response = client.chat.completions.create(
            model=chatgpt_model,
            messages=chat_messages,
            temperature=0  # To encourage deterministic output
        )
        eval_text = chat_response.choices[0].message.content.strip()
        print("Raw evaluation output:", eval_text)

        # Remove markdown code block delimiters if present
        # This regex removes the starting ```json (or ```), and ending ``` from the string.
        eval_text_clean = re.sub(r"^```(?:json)?\s*", "", eval_text).rstrip("`").strip()

        eval_scores = json.loads(eval_text_clean)
    except Exception as e:
        print("Error in evaluation:", e)
        eval_scores = {}
    return eval_scores


###Step 2: Define a Function to Refine the Prompt

In [ ]:
def refine_prompt(insight_text, eval_scores):
    feedback = []
    # Check each criterion and generate feedback if score is below 5.
    if eval_scores.get("Accuracy", 5) < 5:
        feedback.append("improve the accuracy by better identifying the chart type and key data insights")
    if eval_scores.get("Relevance", 5) < 5:
        feedback.append("focus more on the most critical insights and remove unnecessary details")
    if eval_scores.get("Clarity", 5) < 5:
        feedback.append("simplify the language to improve clarity and avoid technical jargon")
    if eval_scores.get("Actionability", 5) < 5:
        feedback.append("suggest clearer, more practical actions")
    if eval_scores.get("Conciseness", 5) < 5:
        feedback.append("make the response less than 50 words")

    # If all scores are perfect, just return the original insight.
    if not feedback:
        return insight_text

    feedback_text = "; ".join(feedback)
    new_prompt = f"Based on the following feedback: {feedback_text}, please refine the following insight: {insight_text}"
    return new_prompt


###Step 3: Create the Iterative Loop

In [ ]:
import pandas as pd
import json

def iterative_insight_process(data_url):
    max_iterations = 5
    target_avg_score = 4.5

    # List to store evaluation records from each iteration
    eval_records = []

    # Generate the initial insight using the data URL-based function.
    current_insight = get_chatgpt_insight_from_dataurl(data_url)
    print("Initial Insight:", current_insight)

    # Store initial insight in the DataFrame
    eval_records.append({
        "Iteration": 0,
        "Insight": current_insight,
        "Average": None,
        "Accuracy": None,
        "Relevance": None,
        "Clarity": None,
        "Actionability": None,
        "Conciseness": None
    })

    for iteration in range(1, max_iterations + 1):
        print(f"\n--- Iteration {iteration} ---")
        eval_scores = evaluate_insight(current_insight)
        if not eval_scores:
            print("Evaluation failed. Exiting iteration.")
            break  # Exit if evaluation returns an empty result

        # Calculate the average score safely
        avg_score = sum(eval_scores.values()) / len(eval_scores) if eval_scores else 0

        # Print each evaluation score individually
        print("Evaluation Scores:")
        for key, value in eval_scores.items():
            print(f"  {key}: {value}")
        print("Average Score:", avg_score)

        # Store iteration data into a dictionary record
        record = {"Iteration": iteration, "Insight": current_insight, "Average": avg_score}
        record.update(eval_scores)
        eval_records.append(record)

        # Stop if target score is met
        if avg_score >= target_avg_score:
            print("Target average score reached. Stopping iterations.")
            break

        # Generate refinement prompt based on evaluation feedback
        refinement_prompt = refine_prompt(current_insight, eval_scores)
        print("Refinement Prompt:", refinement_prompt)

        # Get new insight based solely on the refinement prompt
        chat_messages = [{"role": "user", "content": refinement_prompt}]
        try:
            chat_response = client.chat.completions.create(
                model="gpt-4o-mini",
                messages=chat_messages,
                temperature=0  # Force deterministic output
            )
            current_insight = chat_response.choices[0].message.content.strip()
            print("New Insight:", current_insight)
        except Exception as e:
            print("Error during refinement:", e)
            break

    # Final evaluation for the final insight.
    final_eval_scores = evaluate_insight(current_insight)
    if final_eval_scores:
        final_avg = sum(final_eval_scores.values()) / len(final_eval_scores)
        print("\nFinal Evaluation Scores:")
        for key, value in final_eval_scores.items():
            print(f"  {key}: {value}")
        final_record = {"Iteration": "Final", "Insight": current_insight, "Average": final_avg}
        final_record.update(final_eval_scores)
        eval_records.append(final_record)
    else:
        print("Final evaluation failed, no scores available.")

    # Convert the records to a DataFrame.
    df = pd.DataFrame(eval_records)
    print("\nEvaluation DataFrame:")
    print(df)

    return current_insight, df

# Example usage:
final_insight, evaluation_df = iterative_insight_process(data_url)
print("Final Insight:", final_insight)


Initial Insight: This chart is a "Packaging Control Chart," which tracks the number of late packagings in relation to total orders over time. Here's how to interpret its key elements:

### Axes:
- **X-Axis (Total Order)**: This represents the total number of orders placed.
- **Y-Axis (Late Packaging)**: This measures the incidence of late packagings.

### Trend:
- The overall trend in the chart shows a downward slope, indicating that as the total number of orders increases, the number of late packagings is decreasing. This suggests improvement in the packaging process over time.

### Data Points:
- Each red dot represents a specific measurement of late packagings for corresponding total orders. The scattered nature of the dots above the trend line indicates some variability, even as the overall trend is downward.

### Interpretation:
- A decline in late packagings with an increase in total orders could imply that the packaging process is becoming more efficient or that there are improv